#Activity 1


In [1]:
import math

class Node:
    def __init__(self, state, parent, actions, totalCost, heuristic):
        self.state = state
        self.parent = parent
        self.actions = actions  # List of tuples (neighbor, cost)
        self.totalCost = totalCost
        self.heuristic = heuristic  # (x, y) coordinates for heuristic calculation

def findMin(frontier):
    """Find the node in the frontier with the smallest f value."""
    minVal = math.inf
    node = None
    for n in frontier:
        if frontier[n][1] < minVal:
            minVal = frontier[n][1]
            node = n
    return node

def actionSequence(graph, initialState, goalState):
    """Generate the sequence of actions from initial state to goal state."""
    solution = [goalState]
    currentParent = graph[goalState].parent
    while currentParent is not None:
        solution.append(currentParent)
        currentParent = graph[currentParent].parent
    solution.reverse()
    return solution

def Astar():
    """A* algorithm implementation."""
    initialState = 'A'
    goalState = 'Y'

    # Graph representation with heuristic values as coordinates
    graph = {
        'A': Node('A', None, [('F', 1)], 0, (0, 0)),
        'B': Node('B', None, [('G', 1), ('C', 1)], 0, (2, 0)),
        'C': Node('C', None, [('D', 1)], 0, (3, 0)),
        'D': Node('D', None, [('E', 1)], 0, (4, 0)),
        'E': Node('E', None, [('H', 1)], 0, (5, 0)),
        'F': Node('F', None, [('G', 1), ('I', 1)], 0, (0, 2)),
        'G': Node('G', None, [('J', 1)], 0, (1, 2)),
        'H': Node('H', None, [('K', 1)], 0, (2, 2)),
        'I': Node('I', None, [('L', 1)], 0, (2, 4)),
        'J': Node('J', None, [('M', 1)], 0, (3, 4)),
        'K': Node('K', None, [('N', 1)], 0, (4, 3)),
        'L': Node('L', None, [('O', 1)], 0, (1, 5)),
        'M': Node('M', None, [('P', 1)], 0, (5, 4)),
        'N': Node('N', None, [('Q', 1)], 0, (4, 3)),
        'O': Node('O', None, [('R', 1)], 0, (1, 5)),
        'P': Node('P', None, [('S', 1)], 0, (2, 5)),
        'Q': Node('Q', None, [('T', 1)], 0, (4, 5)),
        'R': Node('R', None, [('U', 1)], 0, (3, 4)),
        'S': Node('S', None, [('V', 1)], 0, (4, 5)),
        'T': Node('T', None, [('W', 1)], 0, (4, 5)),
        'U': Node('U', None, [('X', 1)], 0, (5, 4)),
        'V': Node('V', None, [('Y', 1), ('X', 1)], 0, (4, 5)),
        'W': Node('W', None, [('Y', 1)], 0, (4, 5)),
        'X': Node('X', None, [('V', 1)], 0, (5, 5)),
        'Y': Node('Y', None, [], 0, (5, 5)),
    }

    frontier = dict()
    heuristicCost = math.sqrt(
        (graph[goalState].heuristic[0] - graph[initialState].heuristic[0]) ** 2 +
        (graph[goalState].heuristic[1] - graph[initialState].heuristic[1]) ** 2
    )
    frontier[initialState] = (None, heuristicCost)  # (Parent, f cost)

    explored = dict()

    while len(frontier) != 0:
        currentNode = findMin(frontier)
        print("Exploring:", currentNode)
        del frontier[currentNode]

        if currentNode == goalState:
            return actionSequence(graph, initialState, goalState)

        currentCost = explored[currentNode][1] if currentNode in explored else 0
        explored[currentNode] = (graph[currentNode].parent, currentCost)

        for child in graph[currentNode].actions:
            newCost = currentCost + child[1]
            heuristicCost = math.sqrt(
                (graph[goalState].heuristic[0] - graph[child[0]].heuristic[0]) ** 2 +
                (graph[goalState].heuristic[1] - graph[child[0]].heuristic[1]) ** 2
            )
            totalCost = newCost + heuristicCost

            if child[0] not in explored:
                if child[0] not in frontier or frontier[child[0]][1] > totalCost:
                    graph[child[0]].parent = currentNode
                    frontier[child[0]] = (currentNode, totalCost)

    return None  # No path found

# Run the A* algorithm
solution = Astar()
print("Path from A to Y:", solution)


Exploring: A
Exploring: F
Exploring: I
Exploring: L
Exploring: O
Exploring: R
Exploring: U
Exploring: X
Exploring: V
Exploring: Y
Path from A to Y: ['A', 'F', 'I', 'L', 'O', 'R', 'U', 'X', 'V', 'Y']


#Lab task 1


In [5]:
import heapq

class Node:
    def __init__(self, position, parent=None, g=0, h=0):
        self.position = position  # (x, y) coordinates
        self.parent = parent      # Parent node
        self.g = g                # Cost from start node
        self.h = h                # Heuristic cost to goal
        self.f = g + h            # Total cost

    def __lt__(self, other):
        return self.f < other.f  # Needed for priority queue

def heuristic(a, b):
    """Manhattan distance heuristic."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_neighbors(position, maze):
    """Returns valid neighbors of the current position."""
    x, y = position
    neighbors = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right

    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < len(maze) and 0 <= new_y < len(maze[0]) and maze[new_x][new_y] != 1:
            neighbors.append((new_x, new_y))

    return neighbors

def reconstruct_path(node):
    """Reconstructs the path from start to goal."""
    path = []
    cost = node.g  # The cost of the path
    while node:
        path.append(node.position)
        node = node.parent
    path.reverse()
    return path, cost

def astar(maze, start, goal):
    """A* algorithm to find the shortest path in the maze."""
    open_list = []
    closed_set = set()

    start_node = Node(start, None, 0, heuristic(start, goal))
    heapq.heappush(open_list, start_node)

    while open_list:
        current_node = heapq.heappop(open_list)  # Get node with lowest f cost

        if current_node.position == goal:
            return reconstruct_path(current_node)

        closed_set.add(current_node.position)

        for neighbor in get_neighbors(current_node.position, maze):
            if neighbor in closed_set:
                continue

            new_g = current_node.g + 1  # Each move has a cost of 1
            new_node = Node(neighbor, current_node, new_g, heuristic(neighbor, goal))

            # Check if the node is already in the open list with a better cost
            if any(open_node.position == neighbor and open_node.f <= new_node.f for open_node in open_list):
                continue

            heapq.heappush(open_list, new_node)

    return None, float('inf')  # No path found

# Maze Representation
# 0 = Open path, 1 = Wall, S = Start (Red), G = Goal (Green)
maze = [
    ['S', 0, 1, 0, 0, 0],
    [1, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 1, 1, 0, 1, 0],
    [0, 1, 'G', 0, 0, 0]
]

# Convert maze to numeric format for easier processing
start = None
goal = None
for i in range(len(maze)):
    for j in range(len(maze[0])):
        if maze[i][j] == 'S':
            start = (i, j)
            maze[i][j] = 0
        elif maze[i][j] == 'G':
            goal = (i, j)
            maze[i][j] = 0

# Solve the maze
path, cost = astar(maze, start, goal)

# Output the results
if path:
    print("\nPath followed:", path)
    print("Cost of the path:", cost)
else:
    print("No path found!")



Path followed: [(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (2, 3), (3, 3), (4, 3), (4, 2)]
Cost of the path: 8
